# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [23]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
import glob
import re
from datetime import datetime, timedelta
from pyspark.sql.functions import udf,col,round,dayofweek, weekofyear, year, month
from pyspark.sql.types import StringType, IntegerType

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

##### I94 Immigration Data: 
comes from the U.S. National Tourism and Trade Office and contains various statistics on international visitor arrival in USA and comes from the US National Tourism and Trade Office. The dataset contains data from 2016.

##### U.S. City Demographic Data: 
comes from OpenSoft and contains information about the demographics of all US cities such as average age, male and female 

In [3]:
spark = SparkSession.builder.\
config("spark.jars.repositories", "https://repos.spark-packages.org/").\
config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
enableHiveSupport().getOrCreate()

df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

In [4]:
# Read immigration data
i94_files = glob.glob("../../data/18-83510-I94-Data-2016/*.sas7bdat")
i94_fname = "../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat"
i94_df = spark.read.format("com.github.saurfang.sas.spark").load(i94_fname)

In [5]:
# Read demographics data
demo_fname = "us-cities-demographics.csv"
demo_df = spark.read.format("csv").option("delimiter", ";").option("header", "true").load(demo_fname)

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

##### Demographics Dataset:

In [6]:
# Performing cleaning tasks here
demo_df.count()

2891

In [7]:
demo_df.limit(5).toPandas()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601,41862,82463,1562,30908,2.6,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129,49500,93629,4147,32935,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040,46799,84839,4819,8229,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127,87105,175232,5821,33878,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040,143873,281913,5829,86253,2.73,NJ,White,76402


In [17]:
# Clean demographics data
@udf(StringType())
def city_to_port(city):
    for key in valid_ports:
        if city.lower() in valid_ports[key].lower():
            return key
        
# Calculate percentages of numeric columns about the kind of Population
cleaned_demo_df = demo_df.withColumn("median_age", demo_df['Median Age']) \
    .withColumn("perc_male_pop", (demo_df['Male Population'] / demo_df['Total Population']) * 100) \
    .withColumn("perc_female_pop", (demo_df['Female Population'] / demo_df['Total Population']) * 100) \
    .withColumn("perc_veterans", (demo_df['Number of Veterans'] / demo_df['Total Population']) * 100) \
    .withColumn("perc_foreign_born", (demo_df['Foreign-born'] / demo_df['Total Population']) * 100) \
    .withColumn("perc_race", (demo_df['Count'] / demo_df['Total Population']) * 100) \
    .dropna(how='any', subset=["City"])

cleaned_demo_df = cleaned_demo_df.select(['City', 'State Code', "median_age", "perc_male_pop", 
                                          "perc_female_pop","perc_veterans", "perc_foreign_born", 
                                          "Total Population",'Race','perc_race']).drop_duplicates()

# Pivot the race
pivot_demo_df = cleaned_demo_df.groupBy("City", "State Code", "median_age", "perc_male_pop",
                                        "perc_female_pop","perc_veterans", "perc_foreign_born", "Total Population").pivot("Race").avg("perc_race")

pivot_demo_df = pivot_demo_df.withColumn("city_code", city_to_port(pivot_demo_df["City"])) \
    .dropna(how='any', subset=["city_code"])

# Here we have the demographics stage table
staging_demo_df = pivot_demo_df.select("city_code", "State Code", "City", "median_age",
                                    round(col("perc_male_pop"), 1).alias("perc_male_pop"),
                                    round(col("perc_female_pop"), 1).alias("perc_female_pop"),
                                    round(col("perc_veterans"), 1).alias("perc_veterans"),
                                    round(col("perc_foreign_born"), 1).alias("perc_foreign_born"),
                                    round(col("American Indian and Alaska Native"), 1).alias("perc_native_american"),
                                    round(col("Asian"), 1).alias("perc_asian"),
                                    round(col("Black or African-American"), 1).alias("perc_black"),
                                    round(col("Hispanic or Latino"), 1).alias("perc_hispanic_or_latino"),
                                    round(col("White"), 1).alias("perc_white"), "Total Population")

##### Immigration dataset

In [9]:
i94_df.count()

3096313

In [10]:
i94_df.limit(5).toPandas()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,6.0,2016.0,4.0,692.0,692.0,XXX,20573.0,NaN,None,NaN,...,U,None,1979.0,10282016,None,None,None,1.897628e+09,None,B2
1,7.0,2016.0,4.0,254.0,276.0,ATL,20551.0,1.0,AL,NaN,...,Y,None,1991.0,D/S,M,None,None,3.736796e+09,00296,F1
2,15.0,2016.0,4.0,101.0,101.0,WAS,20545.0,1.0,MI,20691.0,...,None,M,1961.0,09302016,M,None,OS,6.666432e+08,93,B2
3,16.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,1988.0,09302016,None,None,AA,9.246846e+10,00199,B2
4,17.0,2016.0,4.0,101.0,101.0,NYC,20545.0,1.0,MA,20567.0,...,None,M,2012.0,09302016,None,None,AA,9.246846e+10,00199,B2


In [26]:
i94_sas_label_descriptions_fname = "I94_SAS_Labels_Descriptions.SAS"
with open(i94_sas_label_descriptions_fname) as f:
    lines = f.readlines()

re_compiled = re.compile(r"\'(.*)\'.*\'(.*)\'")
valid_ports = {}
for line in lines[302:961]:
    results = re_compiled.search(line)
    valid_ports[results.group(1)] = results.group(2)

valid_states = demo_df.toPandas()["State Code"].unique()

@udf(StringType())
def convert_datetime(x):
    if x:
        return (datetime(1960, 1, 1).date() + timedelta(x)).isoformat()
    return None  

@udf(StringType())
def validate_state(x):  
    if x in valid_states:
        return x
    return 'other'

In [28]:
cleaned_i94_df = i94_df.dropna(how="any", subset=["i94port", "i94addr", "gender"])

# Extract valid states 
cleaned_i94_df = cleaned_i94_df.withColumn("i94addr", validate_state(cleaned_i94_df.i94addr))

# Convert arrival_date to PySpark format
cleaned_i94_df = cleaned_i94_df.withColumn("arrdate", convert_datetime(cleaned_i94_df.arrdate))

# only keep us related immigration data
cleaned_i94_df = cleaned_i94_df.filter(cleaned_i94_df.i94addr != 'other')

staging_i94_df = cleaned_i94_df.select(['cicid', 'arrdate', 'i94port', 'i94addr', 
                                          'i94bir','gender', 'i94visa', 'count']).drop_duplicates()

# Here we have the immigration stage table
staging_i94_df = cleaned_i94_df.select(col("cicid").alias("id"),col("arrdate").alias("date"),
                                       col("i94port").alias("city_code"),col("i94addr").alias("state_code"),
                                       col("i94bir").alias("age"),col("gender").alias("gender"),
                                       col("i94visa").alias("visa_type"),"count")

In [29]:
staging_i94_df.count()

2435922

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

##### The star schema: 
a simple and effective data model. We can query the data by joing fact and dimension tables.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

##### Clean the data: drop duplicates, etc
Load staging tables:
- staging_demo_df
- staging_i94_df
Create dimension tables:
- immigrant_df 
- city_df
- time_df

Create fact table 
- immigration_fact

Save the dimension and fact tables

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [30]:
# Write code here
immigrant_df = staging_i94_df.select("id", "gender", "age", "visa_type").drop_duplicates()

city_df = staging_demo_df \
    .select("city_code", "State Code", "City", "median_age", "perc_male_pop", "perc_female_pop", "perc_veterans",
           "perc_foreign_born", "perc_native_american", "perc_asian", "perc_black",
           "perc_hispanic_or_latino", "perc_white", "Total Population").drop_duplicates()

time_df = staging_i94_df.withColumn("dayofweek", dayofweek("date"))\
                .withColumn("weekofyear", weekofyear("date"))\
                .withColumn("month", month("date")).drop_duplicates()
                        
immigration_fact = staging_i94_df.select("id", "state_code", "city_code", "date", "count").drop_duplicates()

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [31]:
# Perform quality checks here
# Fristly, verify if the table exists
def table_exists(df):
    if df is not None:
        return True
    else:
        return False
        
if table_exists(immigrant_df) & table_exists(city_df) & table_exists(time_df) & table_exists(immigration_fact):
    print("data quality tables OK")
else:
    print("data quality failed: tables do not exist")

data quality tables OK


In [32]:
# Second, verify if the table is empty

def table_not_empty(df):
    return df.count() != 0 

if table_not_empty(immigrant_df) & table_not_empty(city_df) & table_not_empty(time_df) & table_not_empty(immigration_fact):
    print("data quality tables not empty OK")
    print()
else:
    print("data quality check failed: tables empty!")

data quality tables not empty OK



#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

##### immigrant_df
id: id of immigrant  
gender: gender  
age: age  
visa_type: visa type  

##### city_df
city_code: city code  
state_code: state code  
city_name: name of the city  
median_age: median age of the city  
perc_male_pop: city's male in percentage  
perc_female_pop: city's female in percentage  
perc_veterans: city's veteran in percentage  
perc_foreign_born: city's foreign born in percentage  
perc_native_american: city's native american in percentage  
perc_asian: city's asian in percentage  
perc_black: city's black in percentage  
perc_hispanic_or_latino: city's latino population in percentage  
perc_white: city's white population in percentage  
total_pop: city's total population  

##### time_df  
date: date  
dayofweek: day of the week  
weekofyear: week of year  
month: month  
    
##### immigration_fact  
id: id  
state_code: state code  
city_code:  
date: date of arrival  
count: count of immigrant's

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.

##### Apache spark was choosen because of:
- It's manage to handle several file formats and large amounts of data.
- can be unit tested locally and save data to S3.
- easily using APIs for operating on large datasets

##### About the update of the data: 
we need to know the business requirements. 
Currently, I believe that monthly update would be best, considering the subject of the datasets.

##### The data was increased by 100: 
larger and better instances of hosting Spark. if cheaper: additional Spark work nodes.

##### If the data populates a dashboard that must be updated on a daily basis by 7am every day: 
to use Airflow for to schedule, automate and run the pipeline.


##### If the database needed to be accessed by 100+ people:
scale DWH in the cloud, with capacity for users.
In analytics mode, we will move the analytics database into Amazon Redshift, for handle 100+ people.